In [27]:
import pandas as pd
import numpy as np
import statsmodels
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.pyplot as plt

## Question1

In [29]:
sildta = pd.read_csv('data/Reveal_EEO1_for_2016.csv')
sildta = sildta[(sildta.job_category.str.contains('Totals')==False)]
sildta = sildta[(sildta.job_category.str.contains('Previous_totals')==False)]

In [34]:

male_sildta = sildta[sildta['gender']=='male']
jobs = male_sildta.job_category.unique()
model = ols('count~C(job_category, Sum)', data=male_sildta).fit()
table = anova_lm(model, typ=2)

#male_sildta_sen = male_sildta[male_sildta.job_category.str.contains('Senior')]
#male_sildta_nonsen = male_sildta[~male_sildta.job_category.str.contains('Senior')]



ValueError: shapes (1540,242) and (1540,242) not aligned: 242 (dim 1) != 1540 (dim 0)